In [4]:
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec
from pymystem3 import Mystem

In [2]:
word_vectors = KeyedVectors.load_word2vec_format('/data/gensim/news_0_300_2.bin.gz', binary=True)
word_vectors.init_sims(replace=True)

In [5]:
mystem = Mystem()

In [6]:
mystem_part = {
    'A': 'ADJ', # прилагательное
    'ADV': 'ADV', # наречие
    'ADVPRO': 'ADV', # местоименное наречие
    'ANUM': 'ADJ', # числительное-прилагательное
    'APRO': 'DET', # местоимение-прилагательное
    'COM': 'ADJ', # часть композита - сложного слова
    'CONJ': 'SCONJ', # союз
    'INTJ': 'INTJ',	# междометие
    'NUM': 'NUM', # числительное
    'PART': 'PART',	# частица
    'PR': 'ADP', # предлог
    'S': 'NOUN', # существительное
    'SPRO': 'PRON',
    'V': 'VERB', # глагол
}

In [7]:
def lemm(word):
    a = mystem.analyze(word)
    
    if len(a) == 0:
        return word
    
    analysis = a[0]['analysis']
    
    if len(analysis) == 0:
        return word
    
    gr = analysis[0]
    parts = gr['gr'].split("=")
    
    if len(parts) == 0:
        return word
    
    parts2 = parts[0].split(",")
        
    if len(parts2) == 0:
        return word
    
    part = parts2[0]
        
    if part in mystem_part:
        return "%s_%s" % (analysis[0]['lex'], mystem_part[part])
    else:
        return word

In [32]:
def similar(words, lemming=True):
    try:
        w = []
        for word in words:
            if lemming:
                w.append(lemm(word))
            else:
                w.append(word)
        print(w)
        sim = word_vectors.most_similar(positive=w)
        return list(map(lambda x: x[0], sim))
    except Exception as e:
        print(e)
        return words

In [33]:
similar(["президент::сказал"])

['президент_NOUN']


['премьер-министр_NOUN',
 'посол_NOUN',
 'экс-президент_NOUN',
 'посольство_NOUN',
 'президентский_ADJ',
 'генпрокурор_NOUN',
 'министр_NOUN',
 'вице-президент_NOUN',
 'глава_NOUN',
 'мид_NOUN']

In [35]:
word_vectors.most_similar(positive=["президент::сказал_NOUN"])

KeyError: "word 'президент::сказал_NOUN' not in vocabulary"